In [ ]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [ ]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [ ]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=30):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=30):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [ ]:
#read in seattle validation labels
seattle_labels = gpd.read_file('data/round-2/seattle-validation-labels-round2-230608.geojson')
seattle_labels = seattle_labels.drop(columns=['time_created'])
seattle_labels.head()

In [ ]:
#read in chicago validation labels
chicago_labels = gpd.read_file('data/round-2/chicago-validation-labels-round2-230608.geojson')
chicago_labels = chicago_labels.drop(columns=['time_created'])
chicago_labels.head()

In [ ]:
#read in seattle streets
seattle_streets = gpd.read_file("data/round-2/seattle-streets-230608.geojson")
#read in chicago streets
chicago_streets = gpd.read_file("data/round-2/chicago-streets-230608.geojson")

In [ ]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to seattle_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [ ]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [ ]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    return irr

In [ ]:
def calculating_overall_krippendorff_alpha(seattle_labels,chicago_labels,label_type_id):
    seattle = krippendorff_alpha(seattle_labels,label_type_id, seattle_streets, 2285)
    chicago = krippendorff_alpha(chicago_labels,label_type_id, chicago_streets, 3435)
    overall = seattle.append(chicago)
    data_matrix = overall.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')
    return alpha.round(3)


In [ ]:
alpha_overall = {}
for i in [0,1,2,4,5,7,8,9]:
    alpha = calculating_overall_krippendorff_alpha(seattle_labels,chicago_labels,i)
    alpha_overall[label_types[i]] = alpha


In [18]:
alpha_overall

{'CurbRamp': 0.937,
 'NoSidewalk': 0.785,
 'Problem': 0.582,
 'SurfaceProblem': 0.539,
 'Obstacle': 0.554,
 'NoCurbRamp': 0.749,
 'Crosswalk': 0.931,
 'Signal': 0.835}

In [21]:
#take out label type = surfaceproblem and obstacle
seattle_problems= seattle_labels[(seattle_labels['label_type'] == 'SurfaceProblem') | (seattle_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
seattle_problems = seattle_problems[seattle_problems['severity'] >= 4]
seattle_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
107,Devon Snyder,4401,104,Delridge,SurfaceProblem,47.537506,-122.362396,7rtGfmwUxo5KmMttHSV9Ww,None,4.0,f,"cracks,very broken",None,POINT (-122.36240 47.53751)
188,Devon Snyder,4489,128,Delridge,SurfaceProblem,47.545330,-122.362129,Kc8w6pcDom-DMiFu8X6f1g,None,4.0,f,height difference,None,POINT (-122.36213 47.54533)
215,Devon Snyder,4516,126,Delridge,Obstacle,47.541550,-122.360901,aBEy1ylatfdw-tBVOYQnVA,None,4.0,t,"vegetation,narrow",None,POINT (-122.36090 47.54155)
216,Devon Snyder,4517,126,Delridge,Obstacle,47.541466,-122.360878,s84HXlE9_iPAxg_KeDAbeA,None,4.0,t,"vegetation,narrow",None,POINT (-122.36088 47.54147)
219,Devon Snyder,4520,126,Delridge,Obstacle,47.541306,-122.360840,VThyMOgQ5S--MQ23pkrmgA,None,4.0,t,"vegetation,narrow",None,POINT (-122.36084 47.54131)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2820,mikey,7549,653,Rainier Valley,SurfaceProblem,47.545109,-122.267113,5SzoEruKYsDYRb7iXVQyqQ,None,4.0,f,narrow sidewalk,None,POINT (-122.26711 47.54511)
2850,mikey,7579,678,Rainier Valley,Obstacle,47.544842,-122.264229,pvy64U07YbjfB91SqMugMw,None,4.0,f,trash/recycling can,None,POINT (-122.26423 47.54484)
2966,mikey,7704,754,Windermere,Obstacle,47.661606,-122.274765,Vdkot60btTZ1XjhygbDtZw,None,4.0,f,parked bike,None,POINT (-122.27477 47.66161)
3037,mikey,7776,826,Windermere,SurfaceProblem,47.666527,-122.274193,vKQdmFVRV_A4CUd0PagIJw,None,4.0,f,None,None,POINT (-122.27419 47.66653)


In [22]:
#take out label type = surfaceproblem and obstacle
chicago_problems= chicago_labels[(chicago_labels['label_type'] == 'SurfaceProblem') | (chicago_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
chicago_problems = chicago_problems[chicago_problems['severity'] >= 4]
chicago_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
29,Devon Snyder,2579,870,Milton,Obstacle,41.889069,-88.106689,G31WYuAr_xrgi2elpukCwg,None,5.0,t,construction,None,POINT (-88.10669 41.88907)
110,Devon Snyder,2687,940,Naperville,SurfaceProblem,41.801205,-88.153229,Yc7bAyE1oamSSfDM161CYQ,None,4.0,f,"cracks,very broken",None,POINT (-88.15323 41.80120)
233,mikey,2816,1256,Milton,Obstacle,41.889652,-88.106651,W1BXBBCz_AVeEWznKkVPow,None,5.0,f,construction,None,POINT (-88.10665 41.88965)
293,mikey,2877,1014,Milton,Obstacle,41.890877,-88.106155,Ed_OwJO-7XOpuh3iPbLX-w,None,5.0,f,construction,None,POINT (-88.10616 41.89088)
336,mikey,2924,940,Naperville,SurfaceProblem,41.801205,-88.153229,Yc7bAyE1oamSSfDM161CYQ,None,4.0,f,cracks,None,POINT (-88.15323 41.80120)
341,mikey,2930,934,Naperville,SurfaceProblem,41.801243,-88.150719,5z8njsTLwSvNnmnsruxHOA,None,4.0,f,uneven/slanted,None,POINT (-88.15072 41.80124)
556,Devon Snyder,3153,867,Downers Grove North,Obstacle,41.773682,-87.980186,VqXsMyejuv9LSqxptxTVgA,None,5.0,t,vegetation,None,POINT (-87.98019 41.77368)
835,Devon Snyder,3436,1010,Downers Grove North,SurfaceProblem,41.762501,-87.940704,xm8ep910NYSlFJj7z5JKJA,None,4.0,f,height difference,None,POINT (-87.94070 41.76250)
836,Devon Snyder,3437,1010,Downers Grove North,SurfaceProblem,41.762482,-87.940773,xm8ep910NYSlFJj7z5JKJA,None,4.0,f,height difference,None,POINT (-87.94077 41.76248)
888,Devon Snyder,3492,989,Downers Grove North,Obstacle,41.760040,-87.935394,IpAWF-Vq9R_FhPocXbUQHw,None,4.0,t,construction,None,POINT (-87.93539 41.76004)


In [ ]:
calculating_overall_krippendorff_alpha(seattle_problems,chicago_problems,2)

In [23]:
alpha_overall_problems = {}
for i in [2,4,5]:
    alpha = calculating_overall_krippendorff_alpha(seattle_problems,chicago_problems,i)
    alpha_overall_problems[label_types[i]] = alpha

In [24]:
alpha_overall_problems

{'Problem': 0.446, 'SurfaceProblem': 0.21, 'Obstacle': 0.595}